# Examples of CROP
How to use CROP to remove reactions in response to a no-growth phenotype while maintaining other growth phenotypes

![Metabolic-network.jpeg](Metabolic-network.jpeg)

## In this example, we add a hypothetical exchange reaction for B
We then generate phenotypes that suggest it should not be part of the model.


In [14]:
from cobra.util.array import create_stoichiometric_matrix, constraint_matrices
import sys
sys.path.append('../..')
from tests.test_crop import load_model
model = load_model()
stoich_matrix = create_stoichiometric_matrix(model, 'DataFrame')
# Adding an exchange reaction for => B_e
stoich_matrix['EX_B_e'] = 0.0
stoich_matrix['EX_B'] = 0.0
stoich_matrix.loc['B_e'] = 0.0
# Add a transport reaction B_e => B
stoich_matrix.loc['B_e','EX_B_e'] = 1.0
stoich_matrix.loc['B_c','EX_B'] = 1.0
stoich_matrix.loc['B_e','EX_B'] = -1.0

stoich_matrix

,EX_A_e,EX_C_e,EX_D_e,EX_E_e,EX_F_e,EX_A,EX_C,SINK_D,EX_E,SINK_F,R_A_to_B,R_BE_to_D,R_A_to_C,R_B_to_CF,R_C_to_D,EX_B_e,EX_B
A_c,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,-1.0,0.0,-1.0,0.0,0.0,0.0,0.0
B_c,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,-1.0,0.0,-2.0,0.0,0.0,1.0
C_c,0.0,0.0,0.0,0.0,0.0,0.0,-1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,-1.0,0.0,0.0
D_c,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.0,0.0,0.0,0.0,2.0,0.0,0.0,1.0,0.0,0.0
E_c,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,-1.0,0.0,0.0,0.0,0.0,0.0
F_c,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
A_e,-1.0,0.0,0.0,0.0,0.0,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
C_e,0.0,-1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
D_e,0.0,0.0,-1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
E_e,0.0,0.0,0.0,-1.0,0.0,0.0,0.0,0.0,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [16]:
stoich_matrix.shape

(12, 17)

## Defining the phenotypes that exclude EX_B

If we want to create a phenotype that implies we should remove the `EX_B: B_E => B` transport reaction, then we need a phenotype that says, we can grow on `A_e` as a nutrient source, but we cannot grow on `B_e` as a nutrient source.


One issue with the Generic solution strategy is that when there are no solutions that reduce growth to `minimal_growth`, solutions that partially reduce growth are not reported. To overcome this issue, $r_{B}$ can be moved to the objective.

$$\begin{array}{ll}
\min_z &  weights^T(1 - z)  + M\cdot r_{EX_B}\\
 &\begin{array}{lll}
& S^Tm -r = -c \\
& r_i\leq\Omega_i\cdot(1-z_i) \text{ for $i\neq EX_B$} \\
& r_{EX_B} \geq 0 \text{  $B$ is limiting}\\
& r_{EX_A} \geq 0  \text{ $A$ is limiting}\end{array} \\
& Sw = 0 \\
& 0\leq w_A \leq U_{wt,A} \\
& z_A = 1 \\
& 0\leq w_i \leq U_{wt,i}\cdot z_i \\
& w_{D} \geq \text{minimal growth} \\
& w_{F} \geq \text{atp maintenance} \\
& z\in \{0,1\}
\end{array}$$

where the constant $M$ balances
1. the model's consistency with reaction weights versus
2. consistency with growth phenotypes

In [15]:
glucose = 4
ATP = 8

minimal_growth  = 0.1
atp_maintenance = 0.1

B       = 1000
U       = np.eye(nrxns)*1000.0
weights = np.ones(nrxns)*0.5
Omega   = 1000.0

w = xpy.Variable(nrxns)
m = xpy.Variable(nmets)
r = xpy.Variable(nrxns)
z = xpy.Variable(nrxns, boolean=True)

dung_fba_partially_reduced = xpy.Problem(
    xpy.Minimize( weights.T@(1-z) + B*r[glucose]),
        [       S.T@m - r       == -c,
                r[:glucose]     <= Omega*np.eye(glucose)@(1 - z[:glucose]),
                r[glucose]      >= 0,
                r[glucose+1:]   <= Omega*np.eye(nrxns - glucose - 1)@(1 - z[glucose + 1:]),
            S@w             == 0,
            0               <= w,
            w               <= U@z,
            w[biomass]      >= minimal_growth,
            w[ATP]          >= atp_maintenance])
dung_fba_partially_reduced.solve(solver=xpy.SCIPY, )
reactions = pd.DataFrame( z.value, index=rxns, columns=['z'])
reactions['r'] = pd.Series(np.squeeze(np.asarray(r.value)),index=rxns)
reactions['w'] = pd.Series(np.squeeze(np.asarray(w.value)),index=rxns)
display(reactions)
metabolites = pd.DataFrame(m.value,index=internal_mets, columns=['m'])
display(metabolites)

NameError: name 'np' is not defined

In [9]:
import pandas as pd

stoich_matrix

,EX_A_e,EX_C_e,EX_D_e,EX_E_e,EX_F_e,EX_A,EX_C,SINK_D,EX_E,SINK_F,R_A_to_B,R_BE_to_D,R_A_to_C,R_B_to_CF,R_C_to_D,EX_B_e
A_c,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,-1.0,0.0,-1.0,0.0,0.0,0
B_c,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,-1.0,0.0,-2.0,0.0,0
C_c,0.0,0.0,0.0,0.0,0.0,0.0,-1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,-1.0,0
D_c,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.0,0.0,0.0,0.0,2.0,0.0,0.0,1.0,0
E_c,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,-1.0,0.0,0.0,0.0,0
F_c,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.0,0.0,0.0,0.0,1.0,0.0,0
A_e,-1.0,0.0,0.0,0.0,0.0,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1
C_e,0.0,-1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
D_e,0.0,0.0,-1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
E_e,0.0,0.0,0.0,-1.0,0.0,0.0,0.0,0.0,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,0
